-----
<div> <center> ESPACIO PARA BANNER DE LA MAESTRIA </center> </div>   

-----

# XX - Latent Dirichlet Allocation (LDA). 

La asignación latente de Dirichlet (LDA) es una técnica de clustering que puede ser aplicada a colecciones de datos discretos como los son los documentos de texto. LDA es un modelo bayesiano jerárquico de tres niveles en donde cada elemento o palabra de un texto se modela como una mezcla finita de tópicos. A su vez, cada tópico se modela como una combinación infinita de palabras. En este contexto, decimos que LDA es un modelo probabilístico generativo pues estas distribuciones resultan en una representación explícita de cada conjunto de datos o documento.

Esta técnica de aprendizaje no supervisado se diferencia de las técnicas de clustering estudiadas anteriormente en el curso porque en este caso cada observación pertenece a más de un grupo. En ese orden de ideas, se puede decir que LDA es una técnica de agrupamiento difuso (*fuzzy o soft clustering*) donde la pertenencia de un elemento a un grupo se modela como una distribución de probabilidades.

De hecho, la palabra *Dirichlet* dentro del nombre de la técnica hace alusión al Proceso Dirichlet, desarrollado por el matemático alemán Peter Gustav Dirichlet, la cual consiste en una familia de procesos estocásticos en las cuales sus realizaciones son distribuciones de probabilidad. Por otro lado, la palabra *Latante* se refiere a que los tópicos de los que tratan los textos están ocultos o por lo menos no están explícitos.






### Notación
A continuación se hará uso de entidades como "palabras", "documentos" y "corpus" con el fin de guiar la intuición. Sin embargo, eso no quiere decir que el uso de LDA sea exclusivo en textos. Otras posibles aplicaciones de LDA son el filtrado colaborativo, la recuperación de imagenes basadas en contenido y la bioinformática.

Formalmente, se definen los siguientes términos:
- Una *palabra* es la unidad básica de los datos discretos. Definimos una palabra como un item de un vocabulario indexado por $\{1, \cdots, V\}$. Se representan las palabras mediante vectores de base uno, es decir, un vector donde solo un elemento es 1 y el resto son 0. Así, usando superíndices para denotar componentes, la v-ésima palabra en el vocabulario se representa mediante un V-vector $w$ tal que $w^v = 1$ y $w^u = 0$ para $u\neq v$.
- Un *documento* es una secuencia de $N$ palabras denotadas por $\mathbf{w}=(w_1,w_2,\cdots,w_N)$, en donde $w_n$ es la eneava palabra de la secuencia. 
- Un *corpus* es una colección de $M$ documentos dentodas por $\mathbf{D}=(\mathbf{w_1},\mathbf{w_2},\cdots,\mathbf{w_m})$.

Este modelo tiene algunos supuestos como:
- Cada documento es solo una colección de palabras o una "bolsa de palabras". Así, el orden de las palabras y el rol gramatical de las palabras (sujeto, predicado, verbos, ...) no se consideran en el modelo.
- Palabras como a/el/la/pero/y/o/... no contienen ninguna información sobre los "temas" y, por lo tanto, pueden eliminarse de los documentos como un paso de preprocesamiento. De hecho, podemos eliminar palabras que aparecen en al menos %80 ~ %90 de los documentos, sin perder ninguna información. Por ejemplo, si nuestro corpus contiene solo documentos médicos, palabras como humano, cuerpo, salud, etc. pueden estar presentes en la mayoría de los documentos y, por lo tanto, pueden eliminarse ya que no agregan ninguna información específica que haga que el documento se diferencie del resto. 
- Sabemos de antemano cuántos temas queremos. $k$ está predeterminado.
- Los documentos son una combinación de temas.
- Los temas son una combinación de palabras.


### Set-up del modelo
LDA es un modelo probabilístico generativo del *corpus*. La idea general es que los documentos se representan como una combinación aleatoria de los tópicos latentes, donde cada tópico está caracterizado por una distribución sobre las palabras.
LDA supone los siguientes procesos generadores de cada documento $\mathbf(w)$ en el corpus $D$:
1. Escoger $N\sim Poisson(\xi)$.
2. Escoger $\theta\sim Dir(\alpha)$.
3. Para cada palabra $w_n$ de las $N$ palabras:

    a. Escoger un tópico $z_n\sim Multinomial(\theta)$.
    
    b. Escoger una palabra $w_n$ de $p(w_n|z_n, \beta)$, una probabilidad multinomial condicionada en el tópico $z_n$.

Este modelo inicial cuenta con algunas simplificaciones que más adelante serán removidas. En primer lugar, la dimensionalidad $k$ de la distribución Dirichlet (y por consiguiente la dimensionalidad de la variables de los tópicos $z$) se supone fija y conocida. Segundo, las probabilidades de cada palabra son parametrizadas por una matriz $\beta$ de tamaño $k\times V$ en donde $\beta_{ij}=p(w^j = 1|z^i = 1)$, que por ahora trataremos como una cantidad fija que será estimada. Finalmente, el supuesto de $N$ distribuido Poisson no es critico ya que otras distribuciones más realistas sobre la longitud de los documentos pueden ser usadas según la necesidad del investigador. Más aún, note que $N$ es independiente del resto de variables generadoras ($\theta$ y $\mathbf{z}$) por tanto ignoraremos su aleatoriedad en el desarrollo posterior.

Una variable aleatoria $\theta$ de dimensión $k$ puede tomar valores en el $(k-1)-símplex$ (un vector de dimensión $k$ llamado $\theta$ cae en el $(k-1)-símplex$ si $\theta_i\geq0, \sum_{i=1}^k\theta_i=1$), y tiene la siguiente función de densidad en el símplex:

$$p(\theta|\alpha)=\frac{\Gamma(\sum_{i=1}^k\alpha_i)}{\Pi_{i=1}^k\Gamma(\alpha_i)}\theta_1^{(\alpha_1-1)}\cdots \theta_k^{(\alpha_k-1)}$$

En donde el parámetro $\alpha$ es un vector de dimensión $k$ con componentes $a_i$ tal que $a_i\geq0$, y $\Gamma(x)$ es la función Gamma. La Dirichlet es una distribución conveniente para el símplex porque está en la familia exponencial, tiene estadísticas suficientes de dimensión finita y es conjugada a la distribución multinomial.

Antes de continuar, vale la pena profundizar un poco sobre qué es un símplex y por qué es relevante en este modelo. Podemos pensar en el LDA como una aproximación geométrica en donde el símplex es una figura con $k$ vertices equidistantes entre ellos. Un 0-símplex es un punto, un 1-símplex es un segmento de línea, un 2-símplex es un triángulo, un 3-símplex es un tetraedro y un 4-símplex es un pentácoron.

<center>
<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Simplexes.jpg/800px-Simplexes.jpg" alt = "simplex" style = "width: 500px;"/>
</center>

En el caso de LDA, los documentos dentro del corpus se distribuirían a lo largo del símplex en donde cada vértice se representa un tópico. Luego, cada documento se ubicaría más cercano a los vértices que representan los tópicos contenidos en él. Por ejemplo, supongamos que tenemos 7 documentos y tres tópicos posibles (Deporte, Ciencia y Política), podríamos representar los documentos dentro del símplex de la siguiente manera:

<center>
<img src = "data/2-simplex.png" alt = "LDA" style = "width: 500px;"/>
</center>

De esta manera podríamos ver que cada documento es una combinación de tópicos. El documento 1 sería 100% sobre Deportes, el documento 2 sería 50% sobre Deportes y 50% sobre Ciencia, el documento 3 sería 100% sobre Ciencia, etc.

Continuando con nuestra explicación del modelo. Tomando como dados los parámetros $\alpha$ y $\beta$, la distribución de probabilidad conjunta de una mezcla de tópicos $\theta$, un conjunto de $N$ temas $\mathbf{z}$ y un conjunto de $N$ palabras $\mathbf{w}$ es:

$$p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta) = p(\theta|\alpha)\Pi_{n=1}^N p(z_n|\theta)p(w_n|z_n, \beta)$$

En donde $p(z_n|\theta)$ es simplemente $\theta_i$ para el único $i$ en donde $z_n^i=1$. 

A continuación se muestra una representación gráfica del modelo. El rectángulo exterior M representa los documentos y el rectángulo interior N representa la escogencia repetida de palabras ($w$) y tópicos ($z$) al interior de un documento.

<center>
<img src = "data/LDA1.png" alt = "LDA1" style = "width: 500px;"/>
</center>

De la expresión anterior podemos decir que el lado izquierdo de la ecuación ($p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta)$) corresponde a la probabilidad de que un documento $x$ aparezca en nuestro corpus. De este modo, nuestro objetivo será estimar $\alpha$ y $\beta$ de modo que se maximice la probabilidad de encontrar nuestra muestra de documentos.

La Figura anterior corresponde a una ilustración de la ecuación en cuestión. En donde $\alpha$ es una distribución Dirichlet que le asigna una combinación de tópicos a cada uno de los documentos. $\beta$ es una distribución Dirichlet que le asigna una combinación de palabras a cada uno de los tópicos. A partir de $\alpha$ obtenemos $\theta$ la cual es una distribución multinomial de donde se extraen $N$ tópicos. Luego, para cada uno de los tópicos se extrae una palabra de la distribución multinomial que sale de $\beta$.

## Referencias
- Blei, D. M., Jordan, M. I., &; Ng, A. Y. (2003). Latent Dirichlet Allocation. JMLR.org, 3, 993–1022. https://doi.org/10.5555/944919.944937 